In [1]:
import matplotlib as plt
import numpy as np
import pandas as pd
import math
import nltk
import re
import string

### Features for classifiers
(&)in data (+) calculate

#### Query Content:  
&Date of creation of post-User account creation date  
+Count of lowercase chars in body  
+Count of uppercase chars in body  
+lowercase to uppercase ratio in body  
+character count in body  (removed)  
+word count in body  
+code length in body  
+length of first line in body   
+punctuations count  
+sentence count in body  
+count of sentences starting with you  
+count of sentences starting with i  
+count of interrogative sentences  
+count of URLs in body  
+count of stackoverflow URLs in body  
+count of short words in body  
+count of tags  
+tags weight(to be worked upon)  

+character count in title  
+word count in title     
+title starting with what/which/how (useful for opinion based questions) or use interrogative words check


#### User profile

&Reputation  
&View count  
&upvotes received  
&downvotes received   
&about me filled  
&website url filled  
&location filled  
&profile image url filled    
questions asked by user  
answers posted by user  
CloseVotesReceived: The number of close votes received by the user for his posts  
Previous Questions with -ve score  
Previous Questions with +ve score  
Previous Questions with 0 score  
Previous Answers with -ve score  
Previous Answers with +ve score  
Previous Answers with 0 score  

##### URL count
https://codeblog.jonskeet.uk/2010/08/29/writing-the-perfect-question/
Often there will be other people in a similar situation, but the answers didn’t quite match your situation. Just like the above point about unusual constraints, it saves time if you can point out differences between your situation and other common ones. It’s even worth referring to other related questions explicitly – particularly if they’re on the same forum. Aside from anything else, this shows a certain amount of “due diligence” – people are generally more willing to help you if can show you’ve already put some effort in.  

Sharing your research helps everyone. Tell us what you found and why it didn’t meet your needs. This demonstrates that you’ve taken the time to try to help yourself, it saves us from reiterating obvious answers, and above all, it helps you get a more specific and relevant answer!

##### Title length
Write a title that summarizes the specific problem
At the same time a “question” title of “Please help” is unlikely to do well

#### Count of interrogative words
Make sure it’s obvious what you’re trying to get out of the question. Too many “questions” are actually just statements: when I do X, something goes wrong. Well, what did you expect it to do? What are you trying to accomplish? What have you already tried? What happened in those attempts?

#### code length
Shows effort done on the part of asker.  
BUT  
Be as short as possible. If I have to wade through hundreds of lines of code to find the problem, I’m doing work that you should be doing


#### length of first sentence
There’s no need to include greetings and sign-offs such as “Hi everyone!” and “Thanks – hope to get an answer soon” in the question. These will often be edited out by other users, as they’re basically a distraction. Greetings at the start of a question are particularly useless as they can take up valuable space in the snippet displayed in the question list.

##### some closing weight for tags such as homework?

In [53]:
#Resources for text cleaning: 
#https://machinelearningmastery.com/clean-text-machine-learning-python/
#https://github.com/SudalaiRajkumar/NLP/blob/master/src/Text%20Cleaning.ipynb
#https://docs.python.org/3/library/re.html

#Returns number of lowercase chars in the text
def lowercaseCount(text):
    lowercount=0
    for w in text:
        for char in w:
            
            if(char.islower()):
                lowercount+=1
    return lowercount

#Returns number of uppercase chars in the text
def uppercaseCount(text):
    uppercount=0
    for w in text:
        for char in w:
            if(char.isupper()):
                uppercount+=1
    return uppercount
    
#Returns count of total number of words in the query
#https://stackoverflow.com/questions/19410018/how-to-count-the-number-of-words-in-a-sentence-ignoring-numbers-punctuation-an
def countWords(text):
    return len(text.split())

#Removes HTML from query data
#https://stackoverflow.com/questions/3398852/using-python-remove-html-tags-formatting-from-a-string
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)
    
#Sentence tokenization
from nltk import sent_tokenize
#Tokenizes sentences into sentence list
def getSentTokens(sentences):
    return sent_tokenize(sentences)

#Count of sentences in the post body
def getSentenceCount(text):
    return len(split_into_sentences(text))

def getFirstLineLength(text):
    sentences=split_into_sentences(text)
    #print(len(sentences))
    if(len(sentences)==0):
        #print(text)
        return 0
    firstLine=sentences[0]
    return len(firstLine)
    
def countOfInterrogativeSent(text):
    text=text.lower()
    #tokens=text.split() #1D list of words 
    interrogatives=re.findall(r'\b[a-z]*[^.!][?]',text)
    return len(interrogatives)
    
#Word tokenization
from nltk import word_tokenize
#Tokenizes sentence 1D list into words 2D list
def getWordTokens(sentenceList):
    return [word_tokenize(s) for s in sentenceList]

#Punctuations removal
def removePunctuations(word):
    return re.sub(r'\W+', '', word)

#Stop words removal from list of words
from nltk.corpus import stopwords
eng_stop=set(stopwords.words('english'))
#print(eng_stop)
def removeStopwords(text):
    return [word for word in text if word not in eng_stop]

def sentencesStartWithYouCount(text):
    count=0
    text=text.lower()
    text=re.sub('<.*>','',text)
    sentences=split_into_sentences(text)
    for sentence in sentences:
        tokens=re.split('[\s;\',.\-\%]',sentence)
        #print(tokens)
        if(tokens[0]=="you"):
            #print("token0="+tokens[0])
            count+=1
    return count;

def sentencesStartWithICount(text):
    count=0
    text=text.lower()
    sentences=split_into_sentences(text)
    for sentence in sentences:
        tokens=re.split('[\s;\',.\-\%]',sentence)
        #print(tokens)
        if(tokens[0]=="i"):
            #print("token0="+tokens[0])
            count+=1
    return count;

#https://stackoverflow.com/questions/6883049/regex-to-extract-urls-from-href-attribute-in-html-with-python
#https://stackoverflow.com/questions/1374457/find-out-how-many-times-a-regex-matches-in-a-string-in-python
#Returns count of url in a given post
def urlCount(text):
    urls=re.findall(r'https://?|ftp://',text)
    #print(urls)
    return len(urls)

#Returns count of URls pointing to Stack Overflow 
def SOUrlCount(text):
    SOUrls=re.findall(r'https://stackoverflow.com',text)
    #print(SOUrls)
    return len(SOUrls)
    
#Returns the length of total code present in the post
def codeLength(text):
    #make sure you don't strip HTML beforehand
    codes=re.findall(r"<code>(.*?)</code>",text,flags= re.DOTALL)
    #print(codes)
    return len(''.join(codes))

def makeBinary(text):
    if(pd.isnull(text)):
        return 0
    return 1

#https://stackoverflow.com/questions/12628958/remove-small-words-using-python
def countShortWords(text):
    text=striphtml(text)
    shortwords = re.findall(r'\W*\b\w{1,3}\b',text)
    #print(shortwords)
    return len(shortwords)

#Returns the count of tags in the post
#https://stackoverflow.com/questions/11301387/python-regex-first-shortest-match
def countTags(text):
    tagList=re.findall('<.*?>',text)
    #print(tagList)
    return len(tagList)

In [34]:
#Credits: https://stackoverflow.com/questions/4576077/python-split-text-on-sentences

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])" 

def split_into_sentences(text):
    #text=body
    text=striphtml(text)
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    #print(text)
    return sentences

In [6]:
text = ("This's is@ 9 short, sentence. Amazing! I have tried an approach. I am trying this. I'm trying. " 
       "You should suggest some, too. You've done this before?")
text=text.lower()
print(split_into_sentences(text))
print("Lowercase count:"+str(lowercaseCount(text)))
print("Uppercase count:"+str(uppercaseCount(text)))
#print("Lowercase chars to Upper chars ratio:"+str( lowercaseCount(text)/uppercaseCount(text) ))
text=text.lower()
print(text);

["this's is@ 9 short, sentence.", 'amazing!', 'i have tried an approach.', 'i am trying this.', "i'm trying.", 'you should suggest some, too.', "you've done this before?"]
Lowercase count:110
Uppercase count:0
this's is@ 9 short, sentence. amazing! i have tried an approach. i am trying this. i'm trying. you should suggest some, too. you've done this before?


In [12]:
text2=("<p>Are there any free web based service to manage/use your remote computers?</p> "
"<p>I use <a href=\"https://secure.logmein.com/home.asp\" rel=\"nofollow noreferrer\">Logmein</a> free. "
"It solves the purpose. But wondering any other free services available.</p> "
"<p>Heard <a href=\"https://www.copilot.com/\" rel=\"nofollow noreferrer\">Copilot</a> is great but not free!"
       "<a href=\"https://stackoverflow.com/questions/51884419/how-to-paginate-stack-exchange-data-explorer-sede-results\" "
      )
print(urlCount(text2))
print(SOUrlCount(text2))

3
1


In [13]:
text3=("some text <code> something here </code> some text <code>hello"
"world</code>")
print(codeLength(text3))

26


In [14]:
sentences=getSentTokens(text)
print("Sentence count:"+str(len(sentences)))
print(sentences)

Sentence count:7
["this's is@ 9 short, sentence.", 'amazing!', 'i have tried an approach.', 'i am trying this.', "i'm trying.", 'you should suggest some, too.', "you've done this before?"]


In [15]:
tokens=getWordTokens(sentences)
#tokens is 2D list
for sentence in tokens:
    print(sentence)

['this', "'s", 'is', '@', '9', 'short', ',', 'sentence', '.']
['amazing', '!']
['i', 'have', 'tried', 'an', 'approach', '.']
['i', 'am', 'trying', 'this', '.']
['i', "'m", 'trying', '.']
['you', 'should', 'suggest', 'some', ',', 'too', '.']
['you', "'ve", 'done', 'this', 'before', '?']


In [24]:
print(text)
print("Number of sentences starting with you:"+str(sentencesStartWithYouCount(text)) )
print("Number of sentences starting with i:"+str(sentencesStartWithICount(text)) ) 

this's is@ 9 short, sentence. amazing! i have tried an approach. i am trying this. i'm trying. you should suggest some, too. you've done this before?
Number of sentences starting with you:2
Number of sentences starting with i:3


In [15]:
tokens=[removePunctuations(w) for s in tokens for w in s]
tokens=list( filter(None,tokens) )
print(tokens)
print(len(tokens))

['this', 's', 'is', '9', 'short', 'sentence', 'amazing', 'i', 'have', 'tried', 'an', 'approach', 'i', 'am', 'trying', 'this', 'i', 'm', 'trying', 'you', 'should', 'suggest', 'some', 'too', 'you', 've', 'done', 'this', 'before']
29


In [16]:
tokens=removeStopwords(tokens)
print(tokens)
print(len(tokens))

['9', 'short', 'sentence', 'amazing', 'tried', 'approach', 'trying', 'trying', 'suggest', 'done']
10


In [17]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


If HistoryClosedDate is not null and ClosedDate is null
then the post might be protected
PostId=151969

Multiple rows for same PostId=101754 in Off topic closed question database

In [25]:
data=pd.read_csv("OffTopic1.csv")
print(data.head())

   PostId      Id  UserId  OwnerUserId    HistoryClosedDate  \
0  106963  106963       4            4  2018-03-21 07:42:02   
1   33242   33242       5            5  2017-08-12 17:30:34   
2   53802   53802      20           20  2014-02-19 14:01:15   
3      72      72      25           25  2017-04-20 07:24:44   
4   15731   15731      26           26  2015-03-16 01:49:34   

            ClosedDate  PostHistoryTypeId  Comment  PostTypeId  \
0  2018-03-21 07:42:02                 10      102           1   
1  2017-08-12 17:30:34                 10      102           1   
2  2014-02-19 14:01:15                 10      102           1   
3  2017-04-20 07:24:44                 10      102           1   
4  2015-03-16 01:49:34                 10      102           1   

                                                Body  \
0  <p>I'm building the world's simplest library a...   
1  <p>Is there method (other than trial and error...   
2  <p>I'm currently working on a JavaScript tool ...   


In [43]:
#Adding necessary feature columns
#https://stackoverflow.com/questions/40045632/adding-a-column-in-pandas-df-using-a-function

data['lowerUpperRatio']=data['Body'].apply(lowercaseCount)/data['Body'].apply(uppercaseCount)
data['sentenceCount']=data['Body'].apply(getSentenceCount)
data['firstLineLength']=data['Body'].apply(getFirstLineLength)
data['codeLength']=data['Body'].apply(codeLength)
data['urlCount']=data['Body'].apply(urlCount)
data['SOUrlCount']=data['Body'].apply(SOUrlCount)
data['titleLengthInChars']=data['Title'].apply(len)
data['countOfInterrogativeSent']=data['Body'].apply(countOfInterrogativeSent)
data['sentencesStartWithYouCount']=data['Body'].apply(sentencesStartWithYouCount)
data['sentencesStartWithICount']=data['Body'].apply(sentencesStartWithICount)
data['shortWordCount']=data['Body'].apply(countShortWords)

data['websiteUrlFilled']=data['WebsiteUrl'].apply(makeBinary)
data['locationFilled']=data['Location'].apply(makeBinary)
data['aboutMeFilled']=data['AboutMe'].apply(makeBinary)
data['profileImageUrlFilled']=data['ProfileImageUrl'].apply(makeBinary)

print(data.head())

   PostId      Id  UserId  OwnerUserId    HistoryClosedDate  \
0  106963  106963       4            4  2018-03-21 07:42:02   
1   33242   33242       5            5  2017-08-12 17:30:34   
2   53802   53802      20           20  2014-02-19 14:01:15   
3      72      72      25           25  2017-04-20 07:24:44   
4   15731   15731      26           26  2015-03-16 01:49:34   

            ClosedDate  PostHistoryTypeId  Comment  PostTypeId  \
0  2018-03-21 07:42:02                 10      102           1   
1  2017-08-12 17:30:34                 10      102           1   
2  2014-02-19 14:01:15                 10      102           1   
3  2017-04-20 07:24:44                 10      102           1   
4  2015-03-16 01:49:34                 10      102           1   

                                                Body       ...        \
0  <p>I'm building the world's simplest library a...       ...         
1  <p>Is there method (other than trial and error...       ...         
2  <p>I'

In [52]:
data['bodyWordCount']=data['Body'].apply(countWords)
data['tagCount']=data['Tags'].apply(countTags)
print(data.head())

['<api>', '<language-agnostic>', '<amazon>', '<barcode>']
['<html>', '<css>']
['<javascript>', '<dom>']
['<ruby>', '<rdoc>']
['<javascript>', '<html>']
['<api>']
['<vba>']
['<api>', '<wattpad>']
['<c#>', '<outlook>', '<email>', '<message>', '<msg>']
['<c++>', '<linux>', '<ide>']
['<operating-system>', '<windows-xp>', '<windows-vista>']
['<php>', '<apache>', '<xampp>', '<server-configuration>']
['<mysql>', '<database>', '<database-design>', '<diagram>']
['<javascript>', '<ab-testing>']
['<javascript>', '<uglifyjs>']
['<c++>', '<language-agnostic>']
['<pdf>', '<pdf-generation>']
['<java>', '<xml>', '<csv>', '<data-conversion>']
['<cocoa-touch>', '<wwdc>']
['<.net>', '<api>', '<soap>', '<w3c>', '<w3c-validation>']
['<firefox>', '<licensing>', '<firefox-addon>']
['<c>', '<statistics>', '<numbers>']
['<php>', '<email>', '<attachment>', '<phpmailer>', '<swiftmailer>']
['<php>', '<mp3>', '<speex>']
['<c#>', '<.net>', '<code-coverage>']
['<asp.net>', '<windows>', '<iis>', '<hardware>']
['<debu

['<python>', '<r>', '<plot>', '<gnuplot>']
['<git>', '<merge>', '<git-merge>']
['<image-scanner>']
['<debugging>', '<embedded>', '<serial-port>']
['<iphone>', '<google-app-engine>', '<gwt>']
['<jquery>', '<ajax>', '<asp.net-mvc>', '<pagination>']
['<c#>', '<html>', '<rtf>']
['<language-agnostic>', '<artificial-intelligence>', '<heuristics>']
['<opencv>', '<computer-vision>']
['<iphone>', '<documentation>', '<iphone-video>']
['<c++>', '<regex>', '<linux>']
['<api>', '<google-maps-api-3>']
['<.net>', '<asp.net>', '<sql-server>', '<licensing>']
['<agile>', '<scrum>']
['<sql>', '<database>', '<parsing>', '<ddl>']
['<python>', '<user-interface>', '<wxpython>', '<tkinter>']
['<python>', '<parsing>']
['<c#>', '<sql>', '<recursion>', '<polymorphism>']
['<php>', '<ajax>', '<chat>', '<xmpp>']
['<linux>', '<windows>', '<binary>', '<comparison>']
['<linux>', '<apache>', '<open-source>']
['<c++>', '<windows>', '<distributed-computing>', '<distcc>']
['<asp.net>', '<asp.net-mvc>', '<content-managemen

['<open-source>', '<content-management-system>']
['<asp.net-mvc>', '<code-generation>']
['<mono>', '<ironpython>', '<monodevelop>']
['<git>', '<amazon-web-services>', '<github>', '<amazon-s3>', '<deployment>']
['<web-services>']
['<c#>', '<asp.net>', '<refactoring>', '<resharper>']
['<java>', '<templates>']
['<regex>', '<open-source>', '<komodo>']
['<macos>', '<diagram>']
['<node.js>', '<express>', '<jasmine>']
['<wix>', '<installer>', '<windows-installer>', '<installshield>', '<advanced-installer>']
['<unix>', '<merge>', '<debian>', '<disk-partitioning>']
['<mysql>', '<xml>', '<actionscript-3>']
['<objective-c>', '<cocoa>', '<syntax-highlighting>', '<nstextview>']
['<python>', '<ruby>', '<monitoring>', '<dashboard>', '<health-monitoring>']
['<javascript>', '<string>', '<notepad++>']
['<scrum>']
['<java>', '<scala>', '<jvm>', '<gnupg>']
['<java>']
['<automation>', '<uml>', '<svg>']
['<sql-server>', '<database>', '<unit-testing>']
['<javascript>', '<node.js>']
['<java>', '<unit-testing>

['<c++>', '<stack>', '<throw>', '<brainbench>']
['<open-source>']
['<c++>', '<open-source>', '<stocks>', '<trading>']
['<gpl>']
['<c++>', '<dicom>']
['<c++>', '<tcp>', '<client-server>']
['<open-source>', '<donations>']
['<django>', '<open-source>', '<django-models>']
['<java>', '<executable>', '<executable-jar>']
['<iphone>', '<cocoa-touch>', '<cocos2d-iphone>']
['<web-applications>', '<etl>', '<data-import>']
['<jquery>', '<editor>']
['<java>', '<mongodb>', '<bson>']
['<google-app-engine>', '<sdk>', '<open-source>']
['<r>', '<combinations>']
['<java>', '<graph>', '<plot>', '<ascii>', '<histogram>']
['<.net>', '<wcf>', '<licensing>', '<redistributable>', '<wcf-rest-starter-kit>']
['<c#>', '<javascript>', '<relaxng>', '<relaxer>']
['<javascript>', '<jquery>', '<ajax>', '<dhtml>']
['<python>', '<nlp>']
['<qt>', '<lgpl>', '<n900>']
['<database>', '<language-agnostic>', '<translation>', '<vocabulary>']
['<vue.js>', '<typeahead>', '<bootstrap-typeahead>', '<vuejs2>']
['<linux>', '<svn>', '

['<msbuild>']
['<c#>', '<.net>', '<specifications>']
['<c#>', '<decimal>', '<division>']
['<project-management>', '<project-planning>']
['<c>', '<pointers>']
['<sql-server>']
['<html>', '<asp.net-mvc>', '<architecture>']
['<visual-studio>', '<ms-word>', '<documentation>']
['<c#>', '<.net>', '<opc>']
['<php>', '<image-processing>']
['<web-services>', '<geocoding>']
['<concurrency>', '<parallel-processing>']
['<c#>', '<approval-tests>']
['<.net>', '<xml>', '<dependency-injection>', '<inversion-of-control>', '<ioc-container>']
['<asp.net>', '<css>', '<templates>']
['<content-management-system>', '<portal>']
['<seo>', '<reference>']
['<eclipse>', '<eclipse-plugin>', '<metrics>', '<code-metrics>']
['<math>', '<open-source>', '<probability>', '<poker>']
['<javascript>', '<ios>', '<cocoa-touch>', '<uiwebview>']
['<c#>', '<windows-mobile>', '<compact-framework>', '<user-interface>']
['<c++>', '<android>', '<opengl-es>']
['<facebook>', '<user-interface>', '<icons>']
['<amazon-web-services>', '<

...

['<windows>', '<macos>', '<ssh>', '<openssh>', '<ssh-tunnel>']
['<ios>', '<xcode>', '<maintenance>']
['<javascript>', '<jquery>', '<html>', '<css>']
['<twitter-bootstrap>', '<user-interface>', '<twitter-bootstrap-3>', '<mockups>']
['<svg>', '<d3.js>']
['<javascript>']
['<curl>', '<wget>', '<downloading>']
['<ios>', '<cocoa-touch>', '<uitextview>']
['<java>']
['<java>']
['<jquery>', '<css3>', '<ruby-on-rails-4>']
['<java>', '<ios>', '<automated-tests>', '<appium>']
['<ios>', '<nsstring>', '<nsdata>']
['<html>', '<css>', '<css3>', '<css-position>', '<absolute>']
['<android>', '<android-listview>', '<radio-button>']
['<scrollbar>', '<yii2>']
['<c++>', '<algorithm>']
['<r>', '<graph>', '<social-networking>']
['<css>', '<html5>']
['<windows>', '<batch-file>', '<command-line>']
['<android>']
['<linux>', '<opengl>']
['<ios>', '<objective-c>', '<xcode>', '<uitableview>', '<xcode3.2>']
['<ruby>', '<ruby-on-rails-4>', '<nokogiri>']
['<javascript>', '<jquery>', '<html5>', '<css3>']
['<documentati

['<sql>', '<oracle>', '<date-arithmetic>', '<oraclereports>']
['<java>', '<regex>', '<string>']
['<php>', '<wordpress>']
['<iphone>', '<ios>', '<objective-c>']
['<php>', '<mysql>']
['<java>', '<grails>', '<spring-boot>', '<java-8>', '<sdkman>']
['<android>', '<cpu>', '<chipset>']
['<selenium-webdriver>', '<hp-uft>', '<siebel>']
['<html>', '<css>', '<css3>']
['<github>', '<github-api>']
['<mysql>', '<select>']
['<iphone>', '<app-store>']
['<php>', '<forms>']
['<ms-word>']
['<extjs>', '<extjs4>', '<render>', '<extjs4.2>', '<datefield>']
['<php>', '<moodle>', '<multiple-versions>']
['<python>', '<xml>', '<regex>', '<linux>', '<perl>']
['<angularjs>', '<api>', '<web>']
['<javascript>', '<charts>', '<labels>', '<c3>']
['<iphone>', '<cocos2d-iphone>']
['<javascript>', '<html>', '<css>', '<twitter-bootstrap>']
['<c>', '<arrays>']
['<linux>', '<shell>', '<unix>', '<command-line>']
['<excel>', '<vba>', '<vbscript>', '<linkedin>']
['<php>', '<date>', '<time>']
['<java>', '<android>', '<ios>', '<

['<php>', '<mysql>', '<arrays>']
['<javascript>', '<google-chrome>', '<printing>']
['<javascript>', '<jquery>', '<html>']
['<html>', '<css>', '<css3>']
['<python>', '<get>', '<request>', '<python-requests>']
['<php>', '<regex>', '<string>', '<variables>', '<brackets>']
['<html>', '<seo>', '<schema>']
['<javascript>', '<cordova>', '<documentation>']
['<vb.net>', '<file>']
['<php>', '<wordpress>']
['<php>', '<html>']
['<javascript>', '<json>', '<ecmascript-6>']
['<javascript>', '<jquery>', '<html>', '<css>']
['<regex>', '<perl>']
['<html>']
['<java>', '<java-8>', '<guava>']
['<java>', '<android>', '<sqlite>']
['<winforms>', '<devexpress>', '<itextsharp>', '<infragistics>', '<rad-controls>']
['<windows>', '<batch-processing>']
['<ruby>']
['<c#>', '<asp.net>', '<sharepoint>']
['<c#>', '<string>', '<replace>']
['<c>', '<arrays>']
['<java>', '<mysql>', '<licensing>', '<mysql-connector>']
['<java>', '<regex>']
['<android>', '<google-play>', '<privacy-policy>']
['<c#>', '<sql>', '<sql-server>'

['<java>', '<singleton>']
['<c>']
['<android>', '<eclipse>', '<development-environment>']
['<vmware>']
['<windows-7>', '<path>', '<cmd>', '<dos>']
['<c#>', '<asp.net>', '<visual-studio>', '<https>']
['<php>', '<sql>']
['<php>', '<mysql>', '<database>']
['<algorithm>', '<math>', '<modulo>', '<modular-arithmetic>']
['<tomcat>', '<log4j>']
['<python>']
['<python>']
['<java>']
['<string>', '<bash>', '<int>']
['<c++>', '<constructor>', '<operator-overloading>']
['<css>', '<twitter-bootstrap-3>', '<parent-child>']
['<php>', '<string>']
['<php>', '<function>']
['<java>', '<javascript>', '<web-services>', '<encryption>', '<encoding>']
['<c++>', '<algorithm>', '<binary-search-tree>', '<graph-algorithm>']
['<linux>', '<networking>', '<firewall>', '<centos7>']
['<python>', '<object>', '<python-3.x>', '<methods>']
['<php>', '<simple-html-dom>']
['<mysql>', '<linux>', '<bash>', '<shell>', '<batch-file>']
['<linux>', '<bash>', '<shell>', '<ls>']
['<javascript>', '<node.js>', '<web-services>', '<hero

['<sql>', '<oracle10g>']
['<php>']
['<c++>', '<file>']
['<c>', '<struct>']
['<javascript>', '<jquery>', '<if-statement>', '<logical-operators>']
['<php>', '<ajax>', '<codeigniter>']
['<jquery>', '<asp.net>']
['<ios>', '<uicolor>']
['<android>', '<google-play>', '<ads>']
['<linux>', '<linux-device-driver>', '<embedded-linux>', '<archlinux>']
['<php>', '<string>']
['<php>', '<html>', '<mysql>', '<mysqli>']
['<android>', '<ide>', '<cloud>', '<development-environment>']
['<java>']
['<java>']
['<ios>', '<objective-c>', '<nsstring>']
['<ios>', '<unicode>', '<special-characters>', '<cfbundledisplayname>']
['<java>', '<regex>', '<expression>']
['<c#>', '<oop>', '<object-oriented-analysis>']
['<c#>', '<string>', '<reference>']
['<algorithm>', '<combinatorics>']
['<c++>', '<memory>']
['<dll>']
['<c++>', '<linux>', '<file>', '<sync>']
['<java>', '<arrays>', '<sorting>']
['<javascript>', '<php>', '<html>', '<css>']
['<ios>', '<xcode>', '<nsmutablearray>']
['<c++>', '<algorithm>']
['<c#>', '<.net>'

In [27]:
data.columns

Index(['PostId', 'Id', 'UserId', 'OwnerUserId', 'HistoryClosedDate',
       'ClosedDate', 'PostHistoryTypeId', 'Comment', 'PostTypeId', 'Body',
       'Title', 'Tags', 'Reputation', 'PostCreationDate', 'UserCreationDate',
       'experienceInTime', 'WebsiteUrl', 'Location', 'AboutMe', 'Views',
       'Upvotes', 'Downvotes', 'ProfileImageUrl', 'lowerUpperRatio',
       'sentenceCount', 'firstLineLength', 'codeLength', 'urlCount',
       'SOUrlCount', 'titleLength', 'countOfInterrogativeSent',
       'sentencesStartWithYouCount', 'sentencesStartWithICount',
       'websiteUrlFilled', 'locationFilled', 'aboutMeFilled',
       'profileImageUrlFilled'],
      dtype='object')

In [40]:
removeTags=data.iloc[2].Body
print(data.iloc[2].Body)
removeTags=striphtml(removeTags)
print(removeTags)
print(countShortWords(removeTags))
print(split_into_sentences(removeTags))

<p>I'm currently working on a JavaScript tool that, during the course of its execution, will ultimately traverse each node in the DOM. Because this has potential to be a very expensive task, I'd like to benchmark the performance of this script.</p>

<p>What's the best, free tool for benchmarking a script such as this across the major browsers? Ideally, I'd like the tool (or set of tools, even):</p>

<ul>
<li>
**To generate some form of report based on the results of the test.** It can be as simple as a table showing execution times, or as complex as generating some form of a chart. Either way is fine.
</li>
<li>
**To be free.** it's not that I don't believe in paying for software, it's just that I don't have a major need for a tool like this in my typical day-to-day tasks.
</li>
</ul>

<p>If possible, I'd also like the tool to generate varying levels of complex pages so that I can stress test a set of DOMs. This isn't a necessity - if I need to do so, I can write one myself; however, I

In [22]:
text5="I am Ph.D and A.B.c.de. He said, \"Hello!\""
print((text5))
text5 = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text5)
if "Ph.D" in text5: text5 = text5.replace("Ph.D.","Ph<prd>D<prd>")
print((text5))
if "”" in text5: text5 = text5.replace(".”","”.")
print((text5))
if "\"" in text5: text5 = text5.replace(".\"","\".")
if "!" in text5: text5 = text5.replace("!\"","\"!")
print((text5))
if "?" in text5: text5 = text5.replace("?\"","\"?")
text5 = text5.replace(".",".<stop>")
text5 = text5.replace("?","?<stop>")
text5 = text5.replace("!","!<stop>")
text5 = text5.replace("<prd>",".")
print(text5)
sentences = text5.split("<stop>")
print(sentences)

I am Ph.D and A.B.c.de. He said, "Hello!"
I am Ph.D and A<prd>B<prd>c<prd>de. He said, "Hello!"
I am Ph.D and A<prd>B<prd>c<prd>de. He said, "Hello!"
I am Ph.D and A<prd>B<prd>c<prd>de. He said, "Hello"!
I am Ph.<stop>D and A.B.c.de.<stop> He said, "Hello"!<stop>
['I am Ph.', 'D and A.B.c.de.', ' He said, "Hello"!', '']


In [23]:
text6=("Hello everyone! \n"
"I want to ask a question.")
print(text6)
print(getFirstLineLength(text6))
print(text6)

Hello everyone! 
I want to ask a question.
15
Hello everyone! 
I want to ask a question.


In [24]:
re.findall(r'[!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]',"hello! he\ @")

['!', '@']

In [51]:
print(countTags(data.iloc[2].Tags))

['<javascript>', '<dom>']
2
